In [1]:
import numpy as np
import pandas as pd
import csv
import os
import ipynb
import math
import sklearn
import pickle
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score
from scipy.stats import rankdata

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
#import the data from the training csv
#then, firsst encode character attributes into integer attributes for an easier modeling
#3 additional columns are added, indicating the appereace of missing value and the product of attribute_2 and attribute_3


datatest = pd.read_csv(r'C:\Users\user\Desktop\tabular-playground-series-aug-2022\test.csv', delimiter=',', usecols = ['id','product_code','loading','attribute_0','attribute_1','attribute_2','attribute_3','measurement_0','measurement_1','measurement_2','measurement_3','measurement_4','measurement_5','measurement_0','measurement_6','measurement_7','measurement_0','measurement_8','measurement_9','measurement_10','measurement_11','measurement_12','measurement_13','measurement_14','measurement_15','measurement_16','measurement_17'])

cleanup_numst1 = {"attribute_0": {"material_5": 0, "material_7": 1}}
cleanup_numst2 = {"product_code": {"F": 0, "G": 1, "H": 2, "I": 3}}
datatest = datatest.replace(cleanup_numst1)
datatest = datatest.replace(cleanup_numst2)

datatest['m3_missing'] = datatest['measurement_3'].isnull().astype(np.int8)
datatest['m5_missing'] = datatest['measurement_5'].isnull().astype(np.int8)
datatest['attribute_2*3'] = datatest['attribute_2'] * datatest['attribute_3']


datatest.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,m3_missing,m5_missing,attribute_2*3
0,26570,0,119.57,0,material_6,6,4,6,9,6,...,18.070,13.772,13.659,16.825,13.742,17.710,634.612,0,0,24
1,26571,0,113.51,0,material_6,6,4,11,8,0,...,NaN,12.473,17.468,16.708,14.776,14.102,537.037,0,0,24
2,26572,0,112.16,0,material_6,6,4,8,12,4,...,18.086,10.907,13.363,15.737,17.065,16.021,658.995,0,0,24
3,26573,0,112.72,0,material_6,6,4,8,11,10,...,16.163,10.933,15.501,15.667,12.620,16.111,594.301,0,0,24
4,26574,0,208.00,0,material_6,6,4,14,16,8,...,17.849,11.941,16.070,16.183,13.324,17.150,801.044,0,0,24


In [3]:
#we then group the data by product_code
#here we drop some redundent attributes and measurements to perform the first feature selection

productf = datatest[datatest['product_code']==0].drop(['attribute_1', 'attribute_2', 'attribute_3'],axis=1)
productg = datatest[datatest['product_code']==1].drop(['attribute_1', 'attribute_2', 'attribute_3'],axis=1)
producth = datatest[datatest['product_code']==2].drop(['attribute_1', 'attribute_2', 'attribute_3'],axis=1)
producti = datatest[datatest['product_code']==3].drop(['attribute_1', 'attribute_2', 'attribute_3'],axis=1)

impute_f = productf.drop(['id', 'measurement_10', 'measurement_11', 'measurement_12', 'measurement_13', 'measurement_14', 'measurement_15', 'measurement_16'],axis=1)
impute_g = productg.drop(['id', 'measurement_10', 'measurement_11', 'measurement_12', 'measurement_13', 'measurement_14', 'measurement_15', 'measurement_16'],axis=1)
impute_h = producth.drop(['id', 'measurement_10', 'measurement_11', 'measurement_12', 'measurement_13', 'measurement_14', 'measurement_15', 'measurement_16'],axis=1)
impute_i = producti.drop(['id', 'measurement_10', 'measurement_11', 'measurement_12', 'measurement_13', 'measurement_14', 'measurement_15', 'measurement_16'],axis=1)

In [4]:
#the next step is to perform imputation for the missing value
#for measurement17, it has significant correlation with other measurements,
#so I use HuberRegressor to compute the coefficient and fill the missing value in measurement17

test_f = impute_f.dropna(how='any')
huberf_1 = HuberRegressor().fit(test_f.iloc[:,9].values.reshape(-1, 1),test_f.iloc[:,13].values.reshape(-1, 1))
huberf_2 = HuberRegressor().fit(test_f.iloc[:,7].values.reshape(-1, 1),test_f.iloc[:,13].values.reshape(-1, 1))

test_g = impute_g.dropna(how='any')
huberg_1 = HuberRegressor().fit(test_g.iloc[:,9].values.reshape(-1, 1),test_g.iloc[:,13].values.reshape(-1, 1))
huberg_2 = HuberRegressor().fit(test_g.iloc[:,7].values.reshape(-1, 1),test_g.iloc[:,13].values.reshape(-1, 1))
huberg_3 = HuberRegressor().fit(test_g.iloc[:,11].values.reshape(-1, 1),test_g.iloc[:,13].values.reshape(-1, 1))

test_h = impute_h.dropna(how='any')
huberh_1 = HuberRegressor().fit(test_h.iloc[:,8].values.reshape(-1, 1),test_h.iloc[:,13].values.reshape(-1, 1))
huberh_2 = HuberRegressor().fit(test_h.iloc[:,12].values.reshape(-1, 1),test_h.iloc[:,13].values.reshape(-1, 1))

test_i = impute_i.dropna(how='any')
huberi_1 = HuberRegressor().fit(test_i.iloc[:,11].values.reshape(-1, 1),test_i.iloc[:,13].values.reshape(-1, 1))
huberi_2 = HuberRegressor().fit(test_i.iloc[:,5].values.reshape(-1, 1),test_i.iloc[:,13].values.reshape(-1, 1))

C:\Users\user\anaconda3\envs\deeplearning\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\anaconda3\envs\deeplearning\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\anaconda3\envs\deeplearning\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\anaconda3\envs\deeplearning\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when 

In [5]:
#imputing using huberregressor according to productcode

for i in range(len(impute_f)):
    if(math.isnan(impute_f.iloc[i,13])==True and math.isnan(impute_f.iloc[i,9])==True):
        impute_f.iloc[i,13] = huberf_2.predict(impute_f.iloc[i,7].reshape(-1, 1))
        
    elif(math.isnan(impute_f.iloc[i,13])==True and math.isnan(impute_f.iloc[i,9])==False):
        impute_f.iloc[i,13] = huberf_1.predict(impute_f.iloc[i,9].reshape(-1, 1))

        
for i in range(len(impute_g)):
    if(math.isnan(impute_g.iloc[i,13])==True and math.isnan(impute_g.iloc[i,9])==True and math.isnan(impute_g.iloc[i,7])==True):
        impute_g.iloc[i,13] = huberg_3.predict(impute_g.iloc[i,11].reshape(-1, 1))
        
    elif(math.isnan(impute_g.iloc[i,13])==True and math.isnan(impute_g.iloc[i,9])==True):
        impute_g.iloc[i,13] = huberg_2.predict(impute_g.iloc[i,7].reshape(-1, 1))
        
    elif(math.isnan(impute_g.iloc[i,13])==True and math.isnan(impute_g.iloc[i,9])==False):
        impute_g.iloc[i,13] = huberg_1.predict(impute_g.iloc[i,9].reshape(-1, 1))

        
for i in range(len(impute_h)):
    if(math.isnan(impute_h.iloc[i,13])==True and math.isnan(impute_h.iloc[i,8])==True):
        impute_h.iloc[i,13] = huberh_2.predict(impute_h.iloc[i,12].reshape(-1, 1))
        
    elif(math.isnan(impute_h.iloc[i,13])==True and math.isnan(impute_h.iloc[i,8])==False):
        impute_h.iloc[i,13] = huberh_1.predict(impute_h.iloc[i,8].reshape(-1, 1))

        
for i in range(len(impute_i)):
    if(math.isnan(impute_i.iloc[i,13])==True and math.isnan(impute_i.iloc[i,11])==True):
        impute_i.iloc[i,13] = huberi_2.predict(impute_i.iloc[i,5].reshape(-1, 1))
        
    elif(math.isnan(impute_i.iloc[i,13])==True and math.isnan(impute_i.iloc[i,11])==False):
        impute_i.iloc[i,13] = huberi_1.predict(impute_i.iloc[i,11].reshape(-1, 1))
        


In [6]:
#imputing using KNN according to productcode


imputer = KNNImputer(n_neighbors=10, weights="uniform")
impute_f['loading'] = imputer.fit_transform(np.array(impute_f['loading']).reshape(-1,1))

imputer = KNNImputer(n_neighbors=10, weights="uniform")
impute_g['loading'] = imputer.fit_transform(np.array(impute_g['loading']).reshape(-1,1))

imputer = KNNImputer(n_neighbors=10, weights="uniform")
impute_h['loading'] = imputer.fit_transform(np.array(impute_h['loading']).reshape(-1,1))

imputer = KNNImputer(n_neighbors=10, weights="uniform")
impute_i['loading'] = imputer.fit_transform(np.array(impute_i['loading']).reshape(-1,1))



In [7]:
#after imputing, drop unnecessary measurements again to create the final feature selected

impute_f = impute_f.drop(['measurement_3', 'measurement_4', 'measurement_5', 'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9'],axis=1)
impute_g = impute_g.drop(['measurement_3', 'measurement_4', 'measurement_5', 'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9'],axis=1)
impute_h = impute_h.drop(['measurement_3', 'measurement_4', 'measurement_5', 'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9'],axis=1)
impute_i = impute_i.drop(['measurement_3', 'measurement_4', 'measurement_5', 'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9'],axis=1)


In [8]:
#a fuction to perform scaling before feeding it to the model, here i just use StandardScaler

def scaling(data):
    scaler = StandardScaler()    
    select_feature = ['measurement_0','measurement_1', 'measurement_2', 'loading', 'measurement_17','attribute_2*3']
    scaled = scaler.fit_transform(data[select_feature])

    new = data.copy()
    new[select_feature] = scaled
    assert len(data) == len(new)


    return new

In [9]:
#combine the data after preprocessing to produce the final testing data


framest = [impute_f, impute_g, impute_h, impute_i]
test=pd.concat(framest)
X_test = test.drop(['product_code'], axis=1)
X_test = scaling(X_test)

In [10]:
#load the model saved earlier, and compute the predictions on the testing data with a balanced weight on every model

predictions = np.zeros(len(X_test))
for i in range(10):
    model = pickle.load(open('finalized_model' + str(i) + '.sav', 'rb'))
    predictions += model.predict_proba(X_test)[:,1] / 10

predictions

array([0.18599365, 0.1631584 , 0.17514629, ..., 0.14546447, 0.21756607,
       0.16200819])

In [12]:
#import the final submission csv and we write the final prediction on it

dfsub = pd.read_csv(r'C:\Users\user\Desktop\tabular-playground-series-aug-2022\sample_submission.csv', delimiter=',', usecols = ['id','failure'])
dfsub['failure'] = rankdata(predictions)

os.makedirs('finalprojectml', exist_ok=True)
dfsub.to_csv('finalprojectml/109550092_submissions.csv',index=False)  